# Travelling Santas

now closed kaggle competition

with added benchmark TSP instances

In [1]:
source = 'santas'
#source = 'tsp'
#source = 200

num_clusters=30

In [2]:
import pandas as pd
import numpy as np
import random
import time

from tqdm import trange

from numba import njit
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist

import matplotlib.pyplot as plt

In [ ]:
# source is either 'file', or an integer of the number of chimneys to randomly drop
if source == 'santas':

    dirpath = r'C:\Users\user\ML\Optimisation\kaggle-travelling-santa\cities.csv'
    chimneys = pd.read_csv(dirpath, usecols=['X','Y'])
    
else:
    chimneys = pd.DataFrame(columns=['X','Y'])
    for i in range(0,source):
        chimneys = chimneys.append({'X': np.random.randint(low=0, high=5100), 
                                    'Y': np.random.randint(low=0, high=3398)}, 
                                   ignore_index=True)

print(np.max(chimneys['X']), np.max(chimneys['Y']))
chimneys.shape        

In [ ]:
# do a k-means clustering
X = chimneys.values
kmeans = KMeans(n_clusters=num_clusters, random_state=1223)
y_pred = kmeans.fit_predict(X)
centroids = kmeans.cluster_centers_
chimneys['cluster'] = y_pred

adjacency_matrix = np.zeros((num_clusters,num_clusters))
threshold = 50

for i in trange(num_clusters):
    centroids[i][0] = int(centroids[i][0])
    centroids[i][1] = int(centroids[i][1])
    for j in range(i+1,num_clusters):

        points_i = chimneys[chimneys['cluster']==i]
        points_j = chimneys[chimneys['cluster']==j]

        dist = cdist(points_i, points_j)
        min_dist = np.min(dist)
        #print(i,j,min_dist)
        if min_dist < threshold:
            adjacency_matrix[i,j] = 1
            adjacency_matrix[j,i] = 1

plt.scatter(chimneys['X'], chimneys['Y'], c=chimneys['cluster'], cmap='viridis')
plt.scatter(centroids[:,0], centroids[:,1], c='red', s=200, alpha=0.5)
plt.show()

# adjacency_matrix
print(adjacency_matrix)



In [ ]:
# cvc - cluster value counts, cdm - cluster distance matrix
cvc = chimneys['cluster'].value_counts()
cdm = np.zeros((num_clusters,num_clusters))
for i in range(0,num_clusters):
    print(f'Cluster {i} has {cvc[i]} chimneys, and is centred at [{centroids[i][0]}, {centroids[i][1]}]')
    for j in range(i+1,num_clusters):
        cdm[i][j] = np.sqrt((centroids[i][0]-centroids[j][0])**2 + (centroids[i][1]-centroids[j][1])**2)
        cdm[j][i] = cdm[i][j]

In [ ]:
# find the shortest hamiltonian path using 2-opt heuristic around the centroids 
# ensuring that that path is constrained by the adjacency matrix
# noting that the adjacency matrix is symmetric and that the path must start and end at the same point which is 0

# generate a random route constrained by the adjacency matrix beginning and ending at 0
route = np.arange(num_clusters)
print(route)

# calculate the total distance of the route
@njit
def total_distance(route, cdm):
    distance = 0.0
    for i in range(0, len(route)-1):
        distance += cdm[route[i]][route[i+1]]
    #distance += cdm[route[-1]][route[0]]
    return distance

print(total_distance(route, cdm))

# calculate the total distance of the route
@njit
def valid_route_check(route, adjacency_matrix):
    valid = 1
    for i in range(0, len(route)-1):
        if adjacency_matrix[route[i]][route[i+1]] == 0:
            valid = 0
            print('invalid route', route[i], route[i+1])
    if adjacency_matrix[route[-1]][route[0]] == 0:
            valid = 0
            print('invalid route', route[-1], route[0])
    return valid

# 2-opt heuristic constrained by the adjacency matrix
@njit
def two_opt(route, adjacency_matrix, cdm):
    best = route
    improved = True
    while improved:
        improved = False
        for i in range(1, len(route)-2):
            for j in range(i+1, len(route)-1):
                if j-i == 1: continue
                new_route = route.copy()
                new_route[i:j] = route[j-1:i-1:-1]
                if (total_distance(new_route, cdm) < total_distance(best, cdm)) and (adjacency_matrix[new_route[i-1]][new_route[i]] == 1) and (adjacency_matrix[new_route[j]][new_route[j+1]] == 1) and (adjacency_matrix[new_route[-1]][new_route[0]] == 1):
                    best = new_route
                    improved = True
                    route = best

    #print(best)
    return best

#print(two_opt(route, adjacency_matrix, cdm))

# necessary to use the numba typed list - np.insert function is not compatible with Numba
@njit
def insert_integer(arr, value, index):
    # Ensure index is within bounds
    if index < 0 or index > len(arr):
        raise IndexError("Index out of bounds")
    
    # Create a new array with one extra element
    new_arr = np.empty(len(arr) + 1, dtype=arr.dtype)
    
    # Copy the elements before the insertion point
    for i in range(index):
        new_arr[i] = arr[i]
    
    # Insert the new value at the given index
    new_arr[index] = value
    
    # Copy the remaining elements after the insertion point
    for i in range(index, len(arr)):
        new_arr[i + 1] = arr[i]
    
    return new_arr

# necessary to use the numba typed list - np.delete function gives inconsistent results with Numba
@njit
def remove_integer(arr, value):
    # Ensure index is within bounds
    index = np.where(arr==value)[0][0]
    #print(index)
    if index < 0 or index > len(arr):
        raise IndexError("value not found")
    
    # Create a new array with one extra element
    new_arr = np.zeros(len(arr) - 1, dtype=arr.dtype)
    
    # Copy the elements before the insertion point
    for i in range(index):
        new_arr[i] = arr[i]
    
    # Copy the remaining elements after the deletion point
    for i in range(index, len(arr)-1):
        new_arr[i] = arr[i+1]
    
    return new_arr

@njit
def perturb(route, adjacency_matrix, strength=1):
    perturbed = route.copy()
    #print(perturbed)
    for i in range(strength):
        selected_node = np.random.randint(1, len(route)-1)
        
        neighbours = np.where(adjacency_matrix[selected_node]==1)[0]
        next_node = np.random.choice(neighbours)
        next_index = np.where(perturbed==next_node)[0][0]

        #print('selected node', selected_node, 'next node', next_node)
        perturbed = remove_integer(perturbed, next_node)

        selected_index = np.where(perturbed==selected_node)[0][0]
        #print('selected node', selected_node, 'next node', next_node, 'selected index', selected_index)
        
        perturbed = insert_integer(perturbed, next_node, selected_index+1)

    #print(perturbed)
    return perturbed

# perturb the route by selecting a random points and linking it to a neighbour from the adjacency matrix 
@njit
def local_search(route, adjacency_matrix, cdm, max_iter=100):
    
    best = route
    #print('got here -1', route)
    for i in range(0, max_iter):     
        #print('got here 0', route)
        perturbed = perturb(route, adjacency_matrix, 5)   
        #print('got here 1', perturbed)
        new_route = two_opt(perturbed, adjacency_matrix, cdm)
        #print('got here 2', new_route)
        #print(total_distance(new_route, cdm), total_distance(best, cdm))
        if total_distance(new_route, cdm) < total_distance(best, cdm):
            best = new_route
            route = best
    return best

route = local_search(route, adjacency_matrix, cdm)
# route = perturb(route, adjacency_matrix, 1)
# print(total_distance(route, cdm))
# route = two_opt(route, adjacency_matrix, cdm)
print(route)
print(total_distance(route, cdm))  
print(valid_route_check(route, adjacency_matrix))  



In [ ]:
# find the halfway points
halfways = np.zeros(shape=(len(route)-1,2))

for i in range(len(route)-1):
    
    # what's the halfway point between each cluster
    a = route[i]
    b = route[i+1]
    halfways[i][0] = (centroids[a][0] + centroids[b][0]) / 2
    halfways[i][1] = (centroids[a][1] + centroids[b][1]) / 2
    
halfways

In [ ]:
# build the array midpoint
# find the chmineys that belong to each cluster which are in the route closest to the halfway between the two clusters
# give the number of the chimney, the X and Y coordinates, and the cluster number

# find the midpoint between the two centroids
midpoint = []
for i in trange(0,len(route)-1):
    x = route[i]
    y = route[i+1]

    points_x = chimneys[chimneys['cluster']==x]
    points_y = chimneys[chimneys['cluster']==y]

    dist = cdist(points_x, points_y)
    min_dist = np.min(dist)

    for j in range(0,len(points_x)):
        for k in range(0,len(points_y)):
            if dist[j][k] == min_dist:
                midpoint.append([i, points_x.iloc[j]['X'], points_x.iloc[j]['Y'], x, y])
                midpoint.append([i, points_y.iloc[k]['X'], points_y.iloc[k]['Y'], x, y])
                break

midpoint = pd.DataFrame(midpoint, columns=['number','X','Y','cluster1','cluster2'])
midpoint

In [10]:
# midpoints are the x and y coordinates of midpoint
midpoints = np.array(midpoint)[:,1:3]

In [ ]:
fig = plt.figure(figsize=(60,60))
ax = fig.add_subplot(111)

# nodes
plt.scatter(x=chimneys['X'], y=chimneys['Y'], c=chimneys['cluster'])

# centroids
for i in range(len(centroids)):
    plt.plot(centroids[i, 0], centroids[i, 1], marker='x', markersize=20, c='r', label=i)
    ax.text(centroids[i, 0], centroids[i, 1]+25, "%d" %i, ha="center", fontsize=30, fontweight='bold')

#plt.scatter(x=centroids[:, 0], y=centroids[:, 1], marker="x", s=200, linewidths=3, c='r', zorder=10, label='Centroids')
plt.scatter(x=halfways[:, 0], y=halfways[:, 1], marker="o", s=100, linewidths=3, c='b', zorder=10)

# plot the midpoint noting only first two columns for x and y
plt.scatter(x=midpoints[:,0], y=midpoints[:,1], marker="o", s=100, linewidths=3, c='g', zorder=10)

# lines
for i in range(len(route)-1):
    a = route[i]
    b = route[i+1]
    plt.plot([centroids[a][0], centroids[b][0]], [centroids[a][1], centroids[b][1]], marker="1", c='r')
plt.plot([centroids[0][0], centroids[-1][0]], [centroids[0][1], centroids[-1][1]], marker="1", c='r')

for i in range(len(midpoint)-1):
    plt.plot([midpoints[i][0], midpoints[i+1][0]], [midpoints[i][1], midpoints[i+1][1]], marker="1", c='k')
plt.plot([midpoints[0][0], midpoints[-1][0]], [midpoints[0][1], midpoints[-1][1]], marker="1", c='k')

plt.show()

In [ ]:
print(valid_route_check(route, adjacency_matrix))

In [5]:
import numpy as np
from scipy.spatial import distance_matrix

# Generate 100 random nodes on a 2D plane
#np.random.seed(42)
nodes = np.random.randint(0, 100, (1000,2))

# Start the path generation with a defined start and finish node
start_node = 0
nodes[start_node] = [1, 1]
finish_node = 99
nodes[finish_node] = [99, 99]

# Compute the pairwise distance matrix
dist_matrix = distance_matrix(nodes, nodes)

@njit
def greedy_full_path(start_node, remaining_nodes, dist_matrix):
    """ Helper function to compute the full greedy path starting from a given node. """
    path = [start_node]
    current_node = start_node
    remaining_nodes = remaining_nodes.copy()
    remaining_nodes.remove(start_node)
    
    while remaining_nodes:
        nearest_node = min(remaining_nodes, key=lambda node: dist_matrix[current_node][node])
        path.append(nearest_node)
        remaining_nodes.remove(nearest_node)
        current_node = nearest_node
    
    return path

@njit
def total_path_distance(path, dist_matrix):
    """ Helper function to compute the total distance of a given path. """
    return sum(dist_matrix[path[i]][path[i+1]] for i in range(len(path) - 1))

def informed_greedy_path_with_start_finish(start_node, finish_node, dist_matrix, threshold_factor=1.2):
    """ Informed greedy path with start and finish nodes, and dynamic thresholding. """
    # Initialize the path with the start node
    path = [start_node]
    
    # Create a set of all nodes except start and finish
    remaining_nodes = set(range(dist_matrix.shape[0]))
    remaining_nodes.remove(start_node)
    remaining_nodes.remove(finish_node)
    
    current_node = start_node
    best_distance_so_far = float('inf')
    
    # Iterate until all intermediate nodes are visited
    while remaining_nodes:
        best_candidate = None
        best_candidate_distance = float('inf')
        
        # Dynamic threshold: start with a large value, update it as we find better paths
        dynamic_threshold = best_distance_so_far * threshold_factor
        
        # Evaluate all neighbors (remaining nodes)
        for candidate in remaining_nodes:
            # Simulate the full greedy path starting from this candidate
            simulated_path = greedy_full_path(candidate, remaining_nodes, dist_matrix)
            
            # Compute the total distance of this simulated path
            simulated_path_distance = total_path_distance([current_node] + simulated_path, dist_matrix)
            
            # Apply dynamic threshold to prune poor choices
            if simulated_path_distance > dynamic_threshold:
                continue  # Skip this candidate if it exceeds the threshold
            
            # Keep track of the best candidate node based on the path distance
            if simulated_path_distance < best_candidate_distance:
                best_candidate_distance = simulated_path_distance
                best_candidate = candidate
        
        # If we found a valid candidate, choose the best one and update the path
        if best_candidate is not None:
            path.append(best_candidate)
            remaining_nodes.remove(best_candidate)
            current_node = best_candidate
            
            # Update the best distance so far
            best_distance_so_far = best_candidate_distance
        else:
            # If no valid candidate is found (due to thresholding), fall back to nearest neighbor
            nearest_node = min(remaining_nodes, key=lambda node: dist_matrix[current_node][node])
            path.append(nearest_node)
            remaining_nodes.remove(nearest_node)
            current_node = nearest_node
    
    # After visiting all intermediate nodes, finish the path by going to the finish node
    path.append(finish_node)
    
    return path

path = greedy_full_path(start_node, list(range(1000)), dist_matrix)

# Generate the informed greedy path with start and finish nodes
#path = informed_greedy_path_with_start_finish(start_node, finish_node, dist_matrix, threshold_factor=1.2)

print("Generated path:", path)


TypingError: Failed in nopython mode pipeline (step: convert make_function into JIT functions)
[1mCannot capture a constant value for variable 'current_node' as there are multiple definitions present.
[1m
File "..\..\..\AppData\Local\Temp\ipykernel_21240\2553351315.py", line 26:[0m
[1m<source missing, REPL/exec in use?>[0m
[0m

In [ ]:
nodes

In [ ]:
fig = plt.figure(figsize=(60,60))
ax = fig.add_subplot(111)

plt.scatter(x=nodes[0], y=nodes[1])

# # nodes
# for i in range(len(nodes)):
#     plt.plot(nodes[i, 0], nodes[i, 1], marker='x', markersize=20, c='r', label=i)
#     ax.text(nodes[i, 0], nodes[i, 1]+25, "%d" %i, ha="center", fontsize=30, fontweight='bold')

# lines
for i in range(len(path)-1):
    a = path[i]
    b = path[i+1]
    plt.plot([nodes[a][0], nodes[b][0]], [nodes[a][1], nodes[b][1]], marker="1", c='r')


plt.show()